# Storing and Loading Models

https://pytorch.org/tutorials/beginner/saving_loading_models.html

## Init, helpers, utils, ...

In [1]:
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/home/stefan/projects/pytorch_tutorial/.venv/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
from IPython.core.debugger import set_trace

# `state_dict()`

## `nn.Module.state_dict()`
`nn.Module` contain state dict, that maps each layer to the learnable parameters.

In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
model = Net()

In [6]:
model.state_dict()

OrderedDict([('conv1.weight',
              tensor([[[[-0.0716, -0.0885, -0.0418, -0.0174,  0.0306],
                        [-0.0987, -0.0719,  0.0415, -0.0615, -0.0813],
                        [ 0.1020,  0.1077, -0.0484, -0.1027,  0.0212],
                        [-0.0158,  0.0715,  0.1063,  0.0149, -0.0665],
                        [ 0.0862,  0.0699,  0.0826,  0.0582,  0.1031]],
              
                       [[ 0.0377,  0.0997, -0.0155,  0.0759,  0.1136],
                        [-0.0491,  0.0229,  0.0384, -0.0084, -0.0572],
                        [ 0.0543, -0.0504, -0.0309,  0.0171, -0.1097],
                        [ 0.0639, -0.0258,  0.0666, -0.0692, -0.0280],
                        [ 0.0159, -0.0713,  0.0845,  0.0009, -0.0742]],
              
                       [[-0.0562, -0.0505,  0.0886, -0.0612,  0.0182],
                        [ 0.0630, -0.0025, -0.0011, -0.1099,  0.0106],
                        [ 0.0115,  0.0749,  0.0138, -0.0455,  0.0599],
               

In [7]:
def state_dict_info(obj):
    print(f"{'layer':25} shape")
    print("===================================================")
    for k,v in obj.state_dict().items():
        try:
            print(f"{k:25} {v.shape}")
        except AttributeError:
            print(f"{k:25} {v}")

In [8]:
state_dict_info(model)

layer                     shape
conv1.weight              torch.Size([6, 3, 5, 5])
conv1.bias                torch.Size([6])
conv2.weight              torch.Size([16, 6, 5, 5])
conv2.bias                torch.Size([16])
fc1.weight                torch.Size([120, 400])
fc1.bias                  torch.Size([120])
fc2.weight                torch.Size([84, 120])
fc2.bias                  torch.Size([84])
fc3.weight                torch.Size([10, 84])
fc3.bias                  torch.Size([10])


## `nn.Optimizer`

Optimizers also have a a `state_dict`.

In [9]:
optimizer = optim.Adadelta(model.parameters())

In [10]:
state_dict_info(optimizer)

layer                     shape
state                     {}
param_groups              [{'lr': 1.0, 'rho': 0.9, 'eps': 1e-06, 'weight_decay': 0, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}]


In [11]:
optimizer.state_dict()["state"]

{}

In [12]:
optimizer.state_dict()["param_groups"]

[{'lr': 1.0,
  'rho': 0.9,
  'eps': 1e-06,
  'weight_decay': 0,
  'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}]

## Storing and loading `state_dict`

In [13]:
model_file = "model_state_dict.pt"
torch.save(model.state_dict(), model_file)

In [14]:
model = Net()
model.load_state_dict(torch.load(model_file))

<All keys matched successfully>

## Storing and loading the full model

In [15]:
model_file = "model_123.pt"
torch.save(model, model_file)

In [16]:
# Only works if code for `Net` is available right now
model = torch.load(model_file)

# Example Checkpointing
You can store model, optimizer and arbitrary information and reload it.

Example:
```python
torch.save(
    {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch,
        'loss': loss,
    },
    PATH,
)
```

# Exercise
- Find out what is going to be in the `state` variable of the `state_dict` of an optimizer.
- Write your own checkpoint functionality.